In [18]:
import pandas as pd 
import plotly.express as px 
import warnings 
from internproject.constants import ROOT_DATA_DIR

add_lp = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_add_lp.parquet")
remove_lp = pd.read_parquet(
    ROOT_DATA_DIR / "stETH_ETH_remove_lp.parquet"
)
lp_transfers = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_lp_transfers.parquet")

In [19]:
stETH_ETH_apr_df = pd.read_csv(
    ROOT_DATA_DIR
    / 'daily_snapshots/0xDC24316b9AE028F1497c275EB9192a3Ea0f67022_apr_df.csv',
    index_col=0,
    parse_dates=True,
).dropna()

##### Reward rate data

In [20]:
stETH_ETH_incentive_rewards_df = pd.read_parquet(
    ROOT_DATA_DIR / 'raw_pool_data/curve_stETH_ETH/extra_rewards_df.parquet'
)
# quantity of LDO rewards issues per second 
px.line(stETH_ETH_incentive_rewards_df['reward_rate'])

### Individual behavior analysis of LARGEST WHALE

In [21]:
lp_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 00:26:18,0x0000000000000000000000000000000000000000,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a,0.000000,Transfer,11591275,88,148,0x4fdd5de055a85451535ae48f21353f922554d9416903...,2021-01-05 00:26:18
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18
2021-01-05 13:36:06,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,0x182B723a58739a9c974cFDB385ceaDb237453c28,69.999492,Transfer,11594833,50,43,0xacd73e0a596914d36650b57a0bf264cc8a8b63e053a8...,2021-01-05 13:36:06
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47
...,...,...,...,...,...,...,...,...,...
2023-06-21 00:00:11,0x99ac10631F69C753DDb595D074422a0922D9056B,0x182B723a58739a9c974cFDB385ceaDb237453c28,0.003230,Transfer,17524329,349,316,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x182B723a58739a9c974cFDB385ceaDb237453c28,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0.003230,Transfer,17524329,349,318,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11
2023-06-21 00:00:11,0x989AEb4d175e16225E39E87d0D97A3360524AD80,0xF403C135812408BFbE8713b5A23a04b3D48AAE31,0.003230,Transfer,17524329,349,321,0xfa9e275dc3d9f53a90c0c817cb16fa6f328acb696b49...,2023-06-21 00:00:11


In [22]:
null_address = '0x0000000000000000000000000000000000000000'
add_liquidity_transfers = lp_transfers[lp_transfers["from"] == null_address]
withdraw_liquidity_transfers = lp_transfers[lp_transfers["to"] == null_address]

In [23]:
withdraw_liquidity_transfers['value'] = withdraw_liquidity_transfers['value']* -1
warnings.filterwarnings('ignore')
withdraw_liquidity_transfers['value']

timestamp
2021-01-05 17:12:33     -2.000945
2021-01-05 17:46:38   -499.501433
2021-01-05 21:03:54     -0.999754
2021-01-06 03:36:23    -10.037649
2021-01-06 12:17:18     -1.002055
                          ...    
2023-06-20 22:25:35     -0.000000
2023-06-20 22:33:11    -65.033669
2023-06-20 22:40:35    -51.167100
2023-06-20 23:28:11     -0.000000
2023-06-21 00:00:11     -0.003230
Name: value, Length: 27851, dtype: float64

ZAPPER

In [24]:
add_liquidity_transfers['user'] = add_liquidity_transfers['to']
withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']

transfer_df = pd.concat([add_liquidity_transfers, withdraw_liquidity_transfers])
transfer_df.columns

warnings.filterwarnings('ignore')

In [25]:
zapper_address = '0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A'
zappers = lp_transfers[(lp_transfers["to"] == zapper_address) | (lp_transfers["from"] == zapper_address)]
zappers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2022-07-29 04:15:07,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,21.357864,Transfer,15235194,87,202,0xdf78e25f7a44fb1bdffc5ad514dde5fef57c61144b17...,2022-07-29 04:15:07
2022-07-29 04:15:07,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,0x182B723a58739a9c974cFDB385ceaDb237453c28,21.357864,Transfer,15235194,87,207,0xdf78e25f7a44fb1bdffc5ad514dde5fef57c61144b17...,2022-07-29 04:15:07
2022-08-10 03:48:02,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,8.896473,Transfer,15312083,159,285,0xa886bca650082b69f96ec13e1395eb685debb6a6a642...,2022-08-10 03:48:02
2022-08-10 03:48:02,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,0x182B723a58739a9c974cFDB385ceaDb237453c28,8.896473,Transfer,15312083,159,291,0xa886bca650082b69f96ec13e1395eb685debb6a6a642...,2022-08-10 03:48:02
2022-09-15 05:17:15,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,0.967628,Transfer,15537045,54,279,0x6e3cc42f30549d0ddabeb3d6b2c08346327f763e82a0...,2022-09-15 05:17:15
...,...,...,...,...,...,...,...,...,...
2023-06-17 21:18:11,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,0x182B723a58739a9c974cFDB385ceaDb237453c28,18.608794,Transfer,17502157,87,229,0x015d8d951dfd2a2928e80c62b3ce62afaed808f3bd98...,2023-06-17 21:18:11
2023-06-18 11:36:35,0x0000000000000000000000000000000000000000,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,1.017582,Transfer,17506409,78,151,0x2b4b482cef0df603f980fa412ebd479380db35d411b7...,2023-06-18 11:36:35
2023-06-18 11:36:35,0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A,0x182B723a58739a9c974cFDB385ceaDb237453c28,1.017582,Transfer,17506409,78,155,0x2b4b482cef0df603f980fa412ebd479380db35d411b7...,2023-06-18 11:36:35


In [26]:
zappers['user'] = zappers['to']

In [27]:
add_liquidity_transfers['user'] = add_liquidity_transfers['to']
withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']

transfer_df = pd.concat([add_liquidity_transfers, withdraw_liquidity_transfers])
transfer_df.columns

warnings.filterwarnings('ignore')

In [28]:
zappers['user'] = zappers['to']

In [29]:
# Filter out rows matching the zapper address
transfer_df_filtered = transfer_df[~transfer_df.isin(zappers.to_dict('list')).all(1)]
transfer_df_filtered


,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2021-01-05 00:26:18,0x0000000000000000000000000000000000000000,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a,0.000000,Transfer,11591275,88,148,0x4fdd5de055a85451535ae48f21353f922554d9416903...,2021-01-05 00:26:18,0x7EeAC6CDdbd1D0B8aF061742D41877D7F707289a
2021-01-05 11:26:21,0x0000000000000000000000000000000000000000,0x6ab615CF8deCFc488186E54066Fc10589C9293A3,0.160000,Transfer,11594223,141,250,0x16d97e10e582735b4731aaec2823f781edb3355fa9b2...,2021-01-05 11:26:21,0x6ab615CF8deCFc488186E54066Fc10589C9293A3
2021-01-05 13:35:18,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,69.999492,Transfer,11594829,94,97,0x8eca09399384c600131f1f9e1d931246e9129f2e1e11...,2021-01-05 13:35:18,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7
2021-01-05 13:48:47,0x0000000000000000000000000000000000000000,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7,1999.989998,Transfer,11594889,122,123,0x6c6cb813a1cfee7cb6bb241b74a961ede8b70eed5ae3...,2021-01-05 13:48:47,0xc43Db41AA6649DddA4ef0Ef20FD4F16BE43144f7
2021-01-05 14:32:47,0x0000000000000000000000000000000000000000,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0.999754,Transfer,11595091,101,132,0xa3ffd6f2c890037b38c11d64e34e991fa1f31318e873...,2021-01-05 14:32:47,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D
...,...,...,...,...,...,...,...,...,...,...
2023-06-20 22:25:35,0xEa508F82728927454bd3ce853171b0e2705880D4,0x0000000000000000000000000000000000000000,-0.000000,Transfer,17523858,45,95,0x6188d0177bb9a8a4a3bb5feb560477d6ec4642c9fde0...,2023-06-20 22:25:35,0xEa508F82728927454bd3ce853171b0e2705880D4
2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7,0x0000000000000000000000000000000000000000,-65.033669,Transfer,17523896,78,240,0x113260903e90d948871a8a0af2f03156c17781a455ea...,2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7
2023-06-20 22:40:35,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7,0x0000000000000000000000000000000000000000,-51.167100,Transfer,17523933,42,101,0x39af71a3003ae573c6739b9679bede548ea7cc53a6b0...,2023-06-20 22:40:35,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7


In [30]:
# finding cumulative sum 
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()

# sorting from smallest to largest (by user's largest balance at one point in time)
whales_sorted_df = transfer_df.groupby('user').max().sort_values('cumsol_col')
whales_sorted_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,86061.625846,Transfer,14937471,273,587,0xef05f74c37b9b32cbfdaf80933c076b344a00af86bb0...,2022-06-10 08:37:37,105327.329795
0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,60862.175792,Transfer,13188689,269,356,0xff8570fa3c8b4b43055ebc7bd984ab412e68a8295530...,2021-09-09 01:44:23,114142.222772
0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,114348.306873,Transfer,17512484,382,1100,0xf2c693113c6e9693eced243131d5e7dfe43c3148cf74...,2023-06-19 08:04:23,118845.280202


In [31]:
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()

# classifying whales as those who have over 50,000 eth 
whale_df = transfer_df.loc[transfer_df['cumsol_col'] > 50000]


In [32]:
largest_whale = transfer_df[transfer_df['user'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
largest_whale

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2022-09-26 14:36:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389,Transfer,15618180,367,385,0x200853bc39a0b4ac57f3a396d22a39a070686ec5e63a...,2022-09-26 14:36:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118782.299389
2022-09-26 19:08:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0.921205,Transfer,15619528,39,152,0xe62463f53c97cc3542eb72887719022b33172209954b...,2022-09-26 19:08:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118783.220593
2022-09-27 04:24:35,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.909552,Transfer,15622291,162,246,0xfb072e072c6557451effe6a61158eed4360decd747c2...,2022-09-27 04:24:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118785.130146
2022-09-27 13:44:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.903760,Transfer,15625068,172,316,0x21ba8a9e96c6d327fb6df54ff59a683ed8dc418fa2c7...,2022-09-27 13:44:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118787.033906
2022-09-27 23:17:23,0x0000000000000000000000000000000000000000,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,1.905536,Transfer,15627904,76,160,0xa1add66f1689c1ffc90c12e53dd7f4fde42a066ccea3...,2022-09-27 23:17:23,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,118788.939442
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428863,87,247,0xaf7747639152a7093b205d38308052f6fba17716c65c...,2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,107303.607420
2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428867,163,204,0x7183029355114ef096e371caae8dc304e9c21f78cecc...,2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,105303.607420
2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428869,70,133,0xfea33c7d1cf4a21e4269a595047bd551dbd8af65b5da...,2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,103303.607420


#### How many transactions did they take part in? 

In [59]:
added = add_liquidity_transfers[add_liquidity_transfers['to'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
len(added)

185

In [60]:
withdrew = withdraw_liquidity_transfers[withdraw_liquidity_transfers['from'] == '0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3']
len(withdrew)

14

In [61]:
# number of transactions this whale took part in
num_transactions = len(added) + len(withdrew) 
num_transactions

199

In [62]:
# number of transctions for whales w/ 50k+ ETH
top_21_whales  = transfer_df.groupby('user').max().sort_values('cumsol_col').tail(21)
for i, whale in enumerate(top_21_whales.index):
    added = add_liquidity_transfers[add_liquidity_transfers['to'] == whale]
    withdrew = withdraw_liquidity_transfers[withdraw_liquidity_transfers['from'] == whale]
    num_transactions = len(added) + len(withdrew) 
    print(i + 1, ":", whale, num_transactions)

1 : 0x712d0f306956a6A4b4F9319aD9B9de48c5345996 6
2 : 0x2339814f8B162b66087F9C12F7Ff268b7fD498Ea 6
3 : 0x271fbE8aB7f1fB262f81C77Ea5303F03DA9d3d6A 702
4 : 0xD275E5cb559D6Dc236a5f8002A5f0b4c8e610701 12
5 : 0xF9fDc2B5F60355A237deb8BD62CC117b1C907f7b 142
6 : 0xcdE35b62c27d70b279cf7d0AA1212fFA9e938CEf 7
7 : 0xBc79855178842FDBA0c353494895DEEf509E26bB 7
8 : 0xca2C8b7664FA4169bd85DA72A968DaB9b78F5882 2
9 : 0x7cCD3bEfb83154B99C02F4DD5AeC5dD76f1ee0b2 2
10 : 0xA86e412109f77c45a3BC1c5870b880492Fb86A14 5
11 : 0xBC8d100e2c7C1d6BbEfc3128bd5185d226a1976A 9
12 : 0x47cC445c8845F7186A1eaB87Ae5D60CDA69b630c 2
13 : 0x6Cf9AA65EBaD7028536E353393630e2340ca6049 83
14 : 0x3BA21b6477F48273f41d241AA3722FFb9E07E247 20
15 : 0x9026A229b535ecF0162Dfe48fDeb3c75f7b2A7AE 11
16 : 0x25AD8b7cd0738331E65b92736b71A4979691E479 1
17 : 0x12b5c9191E186658841F2431943c47278f68E075 3
18 : 0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1 7
19 : 0xEAA7723633cf598E872D611f5EC50a45b65CBc72 32
20 : 0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3 19

##### How long did this whale stay in the pool? 

In [37]:
whales_sorted_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,86061.625846,Transfer,14937471,273,587,0xef05f74c37b9b32cbfdaf80933c076b344a00af86bb0...,2022-06-10 08:37:37,105327.329795
0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,60862.175792,Transfer,13188689,269,356,0xff8570fa3c8b4b43055ebc7bd984ab412e68a8295530...,2021-09-09 01:44:23,114142.222772
0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,114348.306873,Transfer,17512484,382,1100,0xf2c693113c6e9693eced243131d5e7dfe43c3148cf74...,2023-06-19 08:04:23,118845.280202


In [38]:
whales_sorted_df[whales_sorted_df['cumsol_col'] < 0]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0xbC6433537D8125a72b849b1d777a7E0FfE3DdE02,0xbC6433537D8125a72b849b1d777a7E0FfE3DdE02,0x0000000000000000000000000000000000000000,-0.000523,Transfer,17468383,101,210,0xc50f9a6f6c75f21eb03d618f5493992de4fd5ce65b89...,2023-06-13 03:22:59,-0.000523
0x95B097D903993F54E1afd2D7E534F43d4dd32BD5,0x95B097D903993F54E1afd2D7E534F43d4dd32BD5,0x0000000000000000000000000000000000000000,-0.000009,Transfer,16911337,135,285,0xeeccbc0406c55d79980852f93aa84e221b9582fe4018...,2023-03-26 11:34:59,-0.000521
0xE20ee1D250C8d3e0BD140306121001BfDc7a1201,0xE20ee1D250C8d3e0BD140306121001BfDc7a1201,0x0000000000000000000000000000000000000000,-0.000100,Transfer,13898358,78,139,0xd9b0107373665e1166eebd3b95df6aaf98fe21bd93bd...,2021-12-29 05:34:32,-0.000100


##### There are 1,351 unique users who have a negative balance. We will consider them as not part of the pool anymore. 

In [39]:
whales_sorted_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,0x12b5c9191E186658841F2431943c47278f68E075,86061.625846,Transfer,14937471,273,587,0xef05f74c37b9b32cbfdaf80933c076b344a00af86bb0...,2022-06-10 08:37:37,105327.329795
0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,0x9Cf59Ecde926cc1B5FF1Cb9b247E0FEC627091b1,60862.175792,Transfer,13188689,269,356,0xff8570fa3c8b4b43055ebc7bd984ab412e68a8295530...,2021-09-09 01:44:23,114142.222772
0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,0xEAA7723633cf598E872D611f5EC50a45b65CBc72,114348.306873,Transfer,17512484,382,1100,0xf2c693113c6e9693eced243131d5e7dfe43c3148cf74...,2023-06-19 08:04:23,118845.280202


In [40]:
whales_sorted_df[whales_sorted_df['cumsol_col'] < 0]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,cumsol_col
user,,,,,,,,,,
0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0xf44Ac73b957B28207504Ee2fd5d51eCbfeF7d8fF,0x0000000000000000000000000000000000000000,-80752.602372,Transfer,16038263,0,0,0x2166dcfa7f9a828635963de909c78fd3bee673bcbf13...,2022-11-24 07:11:23,-80752.602372
0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x5AC6A7CAc78197ef80F64D076b8Bfee2574DCd5b,0x0000000000000000000000000000000000000000,-19901.074656,Transfer,15966529,14,34,0xe56afad8c1557b19d87989037131406484c9577bde57...,2022-11-14 06:37:23,-19901.074656
0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0xA0Da148b6CcAde41ab551f8cED537A6c436BAaFa,0x0000000000000000000000000000000000000000,-13000.000000,Transfer,14615872,80,129,0xed4804f8260f468f8bf7dbe24c081e57fa9fe1f117f8...,2022-04-19 13:52:55,-13000.000000
0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x6a522F7ad8Ffe69e2B076e13c73dc698eFbd8044,0x0000000000000000000000000000000000000000,-6457.421531,Transfer,14179288,42,73,0x321427338678b6b2eb4c4a3751cab14e15c763fec7b4...,2022-02-10 16:04:10,-6457.421531
0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x6e1D0fEb04281d1a96DFE5D139941bD86c94712f,0x0000000000000000000000000000000000000000,-4604.795480,Transfer,17379760,73,131,0x44f3fdac141b11e692489ba91ecd070e3cc45139b0aa...,2023-05-31 15:31:47,-4604.795480
...,...,...,...,...,...,...,...,...,...,...
0xbC6433537D8125a72b849b1d777a7E0FfE3DdE02,0xbC6433537D8125a72b849b1d777a7E0FfE3DdE02,0x0000000000000000000000000000000000000000,-0.000523,Transfer,17468383,101,210,0xc50f9a6f6c75f21eb03d618f5493992de4fd5ce65b89...,2023-06-13 03:22:59,-0.000523
0x95B097D903993F54E1afd2D7E534F43d4dd32BD5,0x95B097D903993F54E1afd2D7E534F43d4dd32BD5,0x0000000000000000000000000000000000000000,-0.000009,Transfer,16911337,135,285,0xeeccbc0406c55d79980852f93aa84e221b9582fe4018...,2023-03-26 11:34:59,-0.000521
0xE20ee1D250C8d3e0BD140306121001BfDc7a1201,0xE20ee1D250C8d3e0BD140306121001BfDc7a1201,0x0000000000000000000000000000000000000000,-0.000100,Transfer,13898358,78,139,0xd9b0107373665e1166eebd3b95df6aaf98fe21bd93bd...,2021-12-29 05:34:32,-0.000100


##### There are 1,351 unique users who have a negative balance. We will consider them as not part of the pool anymore. 

In [41]:
# choose random user from this group of people who are not part of the pool anymore
person = '0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A'
add_liquidity_transfers[add_liquidity_transfers["to"] == person]


,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,


In [42]:
withdraw_liquidity_transfers[withdraw_liquidity_transfers["from"] == person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user
timestamp,,,,,,,,,,
2023-03-14 03:13:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000010,Transfer,16823446,53,161,0x358adf18c89be02cfa114e8a316b8d7159cf19429064...,2023-03-14 03:13:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 06:17:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000188,Transfer,16838583,111,276,0x440e9910c26cbd8920a05bd9afa6d79fceb65b97599c...,2023-03-16 06:17:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 06:49:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.000188,Transfer,16838742,68,229,0x999da014b76f2c9238b9a8d4f145b2a2af9924c8f00a...,2023-03-16 06:49:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-16 09:07:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.001323,Transfer,16839420,74,185,0x9bbb863e89c591eed89ebc8867202b5ddc3a89874317...,2023-03-16 09:07:59,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A
2023-03-28 02:07:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A,0x0000000000000000000000000000000000000000,-0.006572,Transfer,16922769,35,32,0xa2add4120a7a7685049c4800873d40f45fd55da0b9f6...,2023-03-28 02:07:47,0x0BcF1E72E4e9B9A5f3c1f51f162Abd1e44A2558A


This doesn't make sense to me because this person ONLY withdrew from the pool and never added liquidity. I noticed this with multiple users with negative balances.

### Whale's balance over time compared to eth reward rate 

In [43]:
# creating cumsol col in transfer_df_filtered
transfer_df_filtered.sort_index(inplace=True)
transfer_df_filtered['cumsol_col'] = transfer_df_filtered.groupby(['user'])['value'].cumsum()

# sorting from smallest to largest (by user's largest balance at one point in time)
whales_sorted_df = transfer_df_filtered.groupby('user').max().sort_values('cumsol_col')
whales_sorted_df

# classifying whales as those who have over 50,000 eth 
whale_df = transfer_df_filtered.loc[transfer_df_filtered['cumsol_col'] > 50000]


In [44]:
# the actual graph of whale balance over time vs. reward rate 
import plotly.graph_objects as go 
from plotly.subplots import make_subplots

# first get balances after 2022
transactions_after_2022 = transfer_df_filtered[transfer_df_filtered['timestamp'] > '2022-01-01']
# then select the top 21 whales
top_21_whales  = transactions_after_2022.groupby('user')[['cumsol_col']].max().sort_values('cumsol_col').tail(21)

for whale in top_21_whales.index:
    print(whale)
    largest_whale = transfer_df_filtered[transfer_df_filtered['user'] == whale]
    largest_whale['date'] = pd.to_datetime(largest_whale['timestamp'].dt.date)

    # to avoid duplicate days, we want the balance over time of 
    largest_whale = largest_whale.groupby('date')[['cumsol_col']].last()
    # resample here reindexes by day and then forward fills the values 
    largest_whale = largest_whale[['cumsol_col']].resample('D').ffill()
    
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    fig.add_trace(
    go.Bar(x=largest_whale.index, y=largest_whale['cumsol_col'].ffill(), name='balance over time'),
    secondary_y=False,
    )
    fig.add_trace(
    go.Scatter(x=stETH_ETH_incentive_rewards_df.index, y=stETH_ETH_incentive_rewards_df['reward_rate_eth'], name='reward rate (eth)'),
    secondary_y=True, 
    )

    fig.update_xaxes(title_text='timestamp')
    fig.update_yaxes(title_text='balance over time', secondary_y=False)
    fig.update_yaxes(title_text='reward rate (eth)', secondary_y=True)
    fig.show()

0xC776207c8cD2A9d585E788AA28096AB803293F55


0x0c2f054331bDBEF366f390A343b3015f008488B3


0xa0c7a5e83552426dAE452469Ca6AdC138640f19e


0xb7d0EaBa9dCB38B3b94D7fa27164d5a5a8ED5812


0x5Ce9b49B7A1bE9f2c3DC2B2A5BaCEA56fa21FBeE


...

0x3BA21b6477F48273f41d241AA3722FFb9E07E247


0x9026A229b535ecF0162Dfe48fDeb3c75f7b2A7AE


0x12b5c9191E186658841F2431943c47278f68E075


0xEAA7723633cf598E872D611f5EC50a45b65CBc72


0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3


### As LP incentives are removed from a specific pool (by the DAO who is paying incentives), what is the (in)elasticity of the TVL?

In [45]:
# merge stETH_ETH_apr_df with incentive_rewards_df 
incentive_rewards_filtered = stETH_ETH_incentive_rewards_df.loc[stETH_ETH_incentive_rewards_df.index.isin(stETH_ETH_apr_df.index)]
merged_df = pd.merge_asof(stETH_ETH_apr_df, stETH_ETH_incentive_rewards_df['reward_rate'], left_index=True, right_index=True)


In [46]:
merged_df_2 = pd.merge_asof(merged_df, stETH_ETH_incentive_rewards_df['reward_rate_eth'], left_index=True, right_index=True)

In [47]:
# elasticity of tvl
merged_df_2['token_supply_shifted'] = merged_df_2['lp_token_total_supply'].shift(-14) 
merged_df_2['reward_rate_eth_shifted'] = merged_df_2['reward_rate_eth'].shift(-14) 

# setting variables 
q2 = merged_df_2['lp_token_total_supply']
q1 = merged_df_2['token_supply_shifted']
p2 = merged_df_2['reward_rate_eth'] 
p1 = merged_df_2['reward_rate_eth_shifted'] 

# elasticity formula 
change_in_supply = (p2 - p1) / ((p2 + p1) / 2) * 100
change_in_price = (q2 - q1) / ((q2 + q1) / 2) * 100
merged_df_2['elasticity'] = change_in_supply / change_in_price
merged_df_2['elasticity']


timestamp
2022-10-10   -4.053774
2022-10-11   -3.866003
2022-10-12   -4.830865
2022-10-13   -6.636972
2022-10-14   -8.249498
                ...   
2023-06-17         NaN
2023-06-18         NaN
2023-06-19         NaN
2023-06-20         NaN
2023-06-21         NaN
Name: elasticity, Length: 255, dtype: float64

Just change in supply and change in price

In [48]:
# scaled appropriately 
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(
go.Scatter(y=change_in_supply, name='change in supply'),
    secondary_y=False,
)

fig.add_trace(
go.Scatter(y=change_in_price, name='change in price'),
    secondary_y=True,
) 

fig.update_xaxes(title_text='timestamp')
fig.update_yaxes(title_text='change in supply', secondary_y=False)
fig.update_yaxes(title_text='change in price', secondary_y=True)

Graph of reward rate eth and elasticity

In [57]:
# scaled appropriately 
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(
go.Scatter(x=merged_df_2.index, y=merged_df_2['elasticity'], name='elasticity'),
    secondary_y=False,
)

fig.add_trace(
go.Scatter(x=merged_df_2.index, y=merged_df_2['reward_rate_eth'], name='reward rate eth'),
    secondary_y=True,
)

fig.update_xaxes(title_text='timestamp')
fig.update_yaxes(title_text='elasticity', secondary_y=False)
fig.update_yaxes(title_text='reward_rate_eth', secondary_y=True)

Just elasticity 

In [50]:
px.line(merged_df_2['elasticity'])

### Aggregate behavior of multiple whales

In [51]:
whale_df

,from,to,value,event,block,transaction_index,log_index,hash,timestamp,user,cumsol_col
timestamp,,,,,,,,,,,
2021-04-10 17:44:19,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,31160.119571,Transfer,12213597,65,82,0xbc2b8f4aa968697352f7ab82f6db0dc30f0eb3b0559d...,2021-04-10 17:44:19,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,56051.962624
2021-04-12 19:07:06,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,34577.303605,Transfer,12227029,0,0,0x277a7d200414999879bd2934b06b7cd144b62d181793...,2021-04-12 19:07:06,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,90629.266229
2021-04-13 17:33:41,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,12772.157944,Transfer,12233123,161,183,0x0236cf92c4cca2673faea6a29d538c75cbbc9447ca05...,2021-04-13 17:33:41,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,103401.424173
2021-04-21 22:35:34,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,13208.519255,Transfer,12286307,6,9,0xf1473c518b2a16f07411ab22a760bc3e88d0bbde1f50...,2021-04-21 22:35:34,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,116609.943428
2021-05-07 13:08:27,0x0000000000000000000000000000000000000000,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,29569.346367,Transfer,12387319,0,0,0xa7f2545100be96c67e71124ce06eda1d9eb5e2aac3c4...,2021-05-07 13:08:27,0xdb5AA12AD695Ef2a28C6CdB69f2BB04BEd20a48e,146179.289795
...,...,...,...,...,...,...,...,...,...,...,...
2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428863,87,247,0xaf7747639152a7093b205d38308052f6fba17716c65c...,2023-06-07 13:42:47,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,107303.607420
2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428867,163,204,0x7183029355114ef096e371caae8dc304e9c21f78cecc...,2023-06-07 13:43:35,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,105303.607420
2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,0x0000000000000000000000000000000000000000,-2000.000000,Transfer,17428869,70,133,0xfea33c7d1cf4a21e4269a595047bd551dbd8af65b5da...,2023-06-07 13:43:59,0x43DfFbF34C06EAf9Cd1F9B4c6848b0F1891434b3,103303.607420


In [52]:
# get the whales you are interested in. Make sure to change this to exclude the wallets that used the zapper
top_whales = transfer_df.groupby('user')[['cumsol_col']].max().sort_values('cumsol_col').tail(15)

# get the subset of transactions they are involved in. Only get adds or withdrawal liquidity
whale_transfers = transfer_df[
    ((transfer_df['to'] == null_address) | (transfer_df['from'] == null_address))
    & (transfer_df['user'].isin(top_whales.index))].copy()
whale_transfers['date'] = pd.to_datetime(whale_transfers['timestamp'].dt.date)
# now we know the balance at each day, with a transaction
cumulative_bal_owned_by_whales_sparse = whale_transfers.sort_index()[['value']].cumsum()
# to make it not sparse we use forward fill
cumulative_bal_owned_by_whales_dense = cumulative_bal_owned_by_whales_sparse.resample('D').ffill()
cumulative_bal_owned_by_whales_dense['date'] = pd.to_datetime(cumulative_bal_owned_by_whales_dense.index)
incentive_rewards_filtered['date'] = pd.to_datetime(incentive_rewards_filtered.index)
whale_df = pd.merge(incentive_rewards_filtered, cumulative_bal_owned_by_whales_dense, on='date', how='left')
whale_df.columns = ['lp_token_total_staked', 'reward_rate', 'block', 'reward_rate_eth',       'date', 'lp_tokens_owned_by_whales']
whale_df

# graphing lp tokens owned by whales vs reward rate
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=whale_df['date'], y=whale_df['lp_tokens_owned_by_whales'].ffill(), name='lp_tokens_owned_by_whales'),
    secondary_y=False,)
fig.add_trace(go.Scatter(x=whale_df['date'], y=whale_df['reward_rate_eth'], name='reward_rate_eth'),
    secondary_y=True, 
    )

fig.update_xaxes(title_text='timestamp')
fig.update_yaxes(title_text='balance over time', secondary_y=False)
fig.update_yaxes(title_text='reward rate (eth)', secondary_y=True)

compare to total balances over time (of ALL users) against reward rate eth 

In [53]:
# Filter out only the transactions involving adds or withdrawal liquidity
whale_transfers = transfer_df[
    ((transfer_df['to'] == null_address) | (transfer_df['from'] == null_address))
].copy()
whale_transfers['date'] = pd.to_datetime(whale_transfers['timestamp'].dt.date)

# Calculate the cumulative balance for all users
cumulative_bal_owned_by_whales_sparse = whale_transfers.groupby(['user', 'date'])['value'].sum().groupby('user').cumsum()
cumulative_bal_owned_by_whales_dense = cumulative_bal_owned_by_whales_sparse.unstack('user')

# Resample incentive_rewards_filtered to match the date range of cumulative_bal_owned_by_whales_dense
incentive_rewards_filtered_resampled = incentive_rewards_filtered.resample('D').ffill()

# Perform a join operation on the two DataFrames using the date as the key
whale_df = incentive_rewards_filtered_resampled.join(cumulative_bal_owned_by_whales_dense)

whale_df

# Graphing lp tokens owned by whales vs reward rate for all users in the pool
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=whale_df.index, y=whale_df['lp_token_total_staked'].ffill(), name='lp_tokens_owned_by_all_users'), secondary_y=False)
fig.add_trace(go.Scatter(x=whale_df.index, y=whale_df['reward_rate_eth'], name='reward_rate_eth'), secondary_y=True)

fig.update_xaxes(title_text='timestamp')
fig.update_yaxes(title_text='balance over time', secondary_y=False)
fig.update_yaxes(title_text='reward rate (eth)', secondary_y=True)

fig.show()


# LP token total supply vs reward rate eth

In [54]:
merged_df

,lp_token_total_supply,lp_token_virtual_price,pool_balance_0,pool_balance_1,block,pool_balance_market_0,pool_balance_market_1,market_eth_tvl,pool_balance_market_portion_0,pool_balance_market_portion_1,...,lp_token_discount_percent,raw_base_apr,raw_fee_apr,raw_incentive_apr,raw_price_apr,tokens,pool_address,upper_bound_nav_percent,name,reward_rate
timestamp,,,,,,,,,,,,,,,,,,,,,
2022-10-10,1.193033e+06,1.052961,540497.246476,715965.757611,15713978.0,540497.246476,712013.580348,1.252511e+06,0.431531,0.568469,...,-0.314548,2.974874,0.000000,2.006373,0.235911,"['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',...",0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,100.0,curve_stETH_ETH,0.771605
2022-10-11,1.191834e+06,1.053044,544672.968748,710599.866558,15721143.0,544672.968748,706740.950473,1.251414e+06,0.435246,0.564754,...,-0.307417,2.954066,0.000000,2.034122,0.230562,"['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',...",0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,100.0,curve_stETH_ETH,0.771605
2022-10-12,1.187409e+06,1.053131,552295.537681,698371.452621,15728307.0,552295.537681,694381.253012,1.246677e+06,0.443014,0.556986,...,-0.319046,2.907704,0.117953,1.940503,0.239284,"['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',...",0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,100.0,curve_stETH_ETH,0.771605
2022-10-13,1.184654e+06,1.053211,551082.494338,696777.003127,15735471.0,551082.494338,693197.039409,1.244280e+06,0.442893,0.557107,...,-0.286888,2.903042,0.000000,1.943023,0.215166,"['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',...",0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,100.0,curve_stETH_ETH,0.771605
2022-10-14,1.179419e+06,1.053299,546032.667027,696429.887963,15742628.0,546032.667027,692020.931829,1.238054e+06,0.441041,0.558959,...,-0.354856,2.912176,0.154494,1.986028,0.266142,"['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',...",0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,100.0,curve_stETH_ETH,0.771605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-17,4.961128e+05,1.074698,264218.180085,268954.074254,17495820.0,264218.180085,268780.314070,5.329985e+05,0.495720,0.504280,...,-0.032590,2.312599,0.000000,0.660071,0.024442,"['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',...",0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,100.0,curve_stETH_ETH,0.106096
2023-06-18,4.949922e+05,1.074760,264403.691969,267594.451427,17502962.0,264403.691969,267270.585601,5.316743e+05,0.497304,0.502696,...,-0.060877,2.289845,0.000000,0.635286,0.045658,"['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',...",0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,100.0,curve_stETH_ETH,0.106096
2023-06-19,4.947796e+05,1.074817,264450.565098,267347.001397,17510088.0,264450.565098,266949.488332,5.314001e+05,0.497649,0.502351,...,-0.074749,2.271010,0.000000,0.652293,0.056062,"['0xEeeeeEeeeEeEeeEeEeEeeEEEeeeeEeeeeeeeEEeE',...",0xDC24316b9AE028F1497c275EB9192a3Ea0f67022,100.0,curve_stETH_ETH,0.106096


In [55]:
px.line(merged_df_2[['reward_rate', 'lp_token_total_supply']])

In [56]:
# scale appropriately 
fig = make_subplots(specs=[[{'secondary_y': True}]])
fig.add_trace(
go.Bar(x=merged_df_2.index, y=merged_df_2['lp_token_total_supply'], name='lp token total supply'),
    secondary_y=False,
)

fig.add_trace(
go.Scatter(x=merged_df_2.index, y=merged_df_2['reward_rate_eth'].rolling(7).mean(), name='reward rate eth'),
    secondary_y=True,
)

fig.update_xaxes(title_text='timestamp')
fig.update_yaxes(title_text='lp token total supply', secondary_y=False)
fig.update_yaxes(title_text='reward rate eth', secondary_y=True)
fig.update_layout(title='LP Token Total Supply vs. Reward Rate ETH ')